In [8]:

import os 
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3" # "0,1,2,3" # "0"

In [52]:

import numpy as np
import h5py
import cv2
import os, sys, gc
import argparse
from glob import glob
from core.get_args import get_args
from core.produce_denoised_img import produce_denoised_img_no_crop
from core.utils import TedataLoader
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from core.watershed import watershed,watershed_per_img,watershed_original
from core.median_filter import apply_median_filter_cpu, apply_median_filter_gpu, apply_median_filter_gpu_simple
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:

img_dict = {}
# data_path = "/mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/Samsung_SNU_1474x3010_aligned_ordered/" 
debug = True
path_list = ["/home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216"]
for data_path in path_list:
    print("=====",data_path, "=====")
    for set_num in sorted(os.listdir(data_path)):
        if set_num[0] == '.' :
            continue
        set_path = os.path.join(data_path,set_num)
        set_number = set_num.split(" ")[-1][:-1]
        set_num = f"{set_num[1:4]}{set_number}"
        print("=====",set_num, "=====")
        img_dict[set_num] = {}
        # if debug is True:
        #     print(set_num)
        f_num_list = os.listdir(set_path)
        print(f_num_list)
        for f_num in f_num_list:
            f_path = os.path.join(set_path,f_num)
            f_number = int(f_num[1:])
            f_num = f"F{f_number:02d}"
            # print(set_num,f_num)
            if f_num != 'F08' :
                continue
            # img_dict[set_num][f_num] = None
            img_dict[set_num][f_num] = {}
            img_list = sorted(os.listdir(f_path))
            img_list = list(filter(lambda x : ".ipynb_checkpoints" not in x,img_list))
            
            for filename in sorted(img_list):
                
                img_path = os.path.join(f_path,filename)
                image_idx = int(filename.split("_")[0])
                filename = f"{set_num}_{f_num}_{image_idx:02d}.png"
                if debug is True:
                    print(f_num," & ",filename)
                img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE) 
                # img_dict[set_num][f_num].append(img)
                # img = np.expand_dims(img,axis=0)
                img = np.expand_dims(img,axis=0) / 255.
                img_dict[set_num][f_num][filename] = img
                # if img_dict[set_num][f_num] is None:  
                #     img_dict[set_num][f_num] = img
                # else :
                #     img_dict[set_num][f_num] = np.append(img_dict[set_num][f_num],img,axis=0)
                if debug is True:
                    print(set_num,f_num,img.shape)
                    
if debug is True:
    print(img_dict.keys())
    for set_num in img_dict.keys():
        print(img_dict[set_num].keys())
        for f_num in img_dict[set_num].keys():
            print(img_dict[set_num][f_num].shape)

===== /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216 =====
===== SET1-1 =====
['F32', 'F8', 'F64', 'F16']
F08  &  SET1-1_F08_10.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_11.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_12.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_13.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_14.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_15.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_16.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_17.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_18.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_19.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_01.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_20.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_21.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_22.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_23.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1-1_F08_24.png
SET1-1 F08 (1, 2048, 3072)
F08  &  SET1

AttributeError: 'dict' object has no attribute 'shape'

In [54]:

args = get_args()
args.pge_weight_dir = None #pge_weight_dir
args.loss_function = "MSE_Affine"
args.noise_type = 'Poisson-Gaussian'
args.model_type = 'FBI_Net'
# args.set_num = '1'
args.data_name = 'Samsung'
args.data_type = 'Grayscale'
args.lr = 0.001
args.num_layers = 17
args.num_filters = 64
# args.crop_size = 200
args.debug = False

In [55]:
sorted(glob("../weights/230303_FBI_Net*F08*F16_MSE*"))

['../weights/230303_FBI_Net_Grayscale_Samsung_SET050607080910individual_x_as_F08_y_as_F16_MSE_Affine_layers_x17_filters_x64_cropsize_256.w']

In [66]:
debug= True
# for fbi_weight_dir in sorted(glob("../weights/230207_FBI_Net*")):
for fbi_weight_dir in sorted(glob("../weights/230303_FBI_Net*F08*F16_MSE*")):
    target_y = fbi_weight_dir.split("_y_as_")[1][:3]
    if "x_as_" in fbi_weight_dir:
        target_x = fbi_weight_dir.split("_x_as_")[1][:3]
    else :
        target_x = "F#"
    dataset_version = 'v1' if "with" in fbi_weight_dir else 'v2'

    if target_y not in ['F08','F16']:
        continue
    print("======",target_x,"vs",target_y,dataset_version,f" ======")

    if debug is True:
        print(fbi_weight_dir)
        # print(median_filter_input,"median_filter_input")
    folder_name = f"./denoised_img_{target_x}_vs_{target_y}_{dataset_version}_crushed"

    os.makedirs(folder_name,exist_ok=True)
    
    
    model = produce_denoised_img_no_crop(_pge_weight_dir=None,_fbi_weight_dir = fbi_weight_dir,_args = args)
    
    # DENOISE IMAGE
    denoised_img_dict = {}
    for set_num in img_dict.keys():
        denoised_img_dict[set_num] = {}
        for f_num in img_dict[set_num].keys():
            denoised_img_dict[set_num][f_num] = None
            if f_num != 'F08':
                continue
            for idx,(filename, img) in enumerate(img_dict[set_num][f_num].items()):
                img = np.expand_dims(img, axis=0) # 1,1536x3074
                print(filename,set_num,f_num,idx,img.shape)
            
                denoised_img = model.eval(img)[0]
                # print("after denoised",denoised_img.shape)
                if denoised_img_dict[set_num][f_num] is None:
                    denoised_img_dict[set_num][f_num] = denoised_img
                else :
                    denoised_img_dict[set_num][f_num] = np.append(denoised_img_dict[set_num][f_num], denoised_img,axis=0)
                # filename = f"{set_num}_{f_num}_{idx}.png"
                print("imwrite : ",f"{folder_name}/{filename}")
                cv2.imwrite(f"{folder_name}/{filename}",denoised_img[0]*255)
                # apply median filter after denoising
                
            print(f"Denoising SET : {set_num}, f_num : {f_num} End")
            
    # np.save(f"./intermediate_result/{folder_name}_denoised_img_dict.npy",denoised_img_dict)

    gc.collect()


====== F08 vs F16 v2  ======
../weights/230303_FBI_Net_Grayscale_Samsung_SET050607080910individual_x_as_F08_y_as_F16_MSE_Affine_layers_x17_filters_x64_cropsize_256.w
SET1-1_F08_10.png SET1-1 F08 0 (1, 1, 2048, 3072)
imwrite :  ./denoised_img_F08_vs_F16_v2_crushed/SET1-1_F08_10.png
SET1-1_F08_11.png SET1-1 F08 1 (1, 1, 2048, 3072)
imwrite :  ./denoised_img_F08_vs_F16_v2_crushed/SET1-1_F08_11.png
SET1-1_F08_12.png SET1-1 F08 2 (1, 1, 2048, 3072)
imwrite :  ./denoised_img_F08_vs_F16_v2_crushed/SET1-1_F08_12.png
SET1-1_F08_13.png SET1-1 F08 3 (1, 1, 2048, 3072)
imwrite :  ./denoised_img_F08_vs_F16_v2_crushed/SET1-1_F08_13.png
SET1-1_F08_14.png SET1-1 F08 4 (1, 1, 2048, 3072)
imwrite :  ./denoised_img_F08_vs_F16_v2_crushed/SET1-1_F08_14.png
SET1-1_F08_15.png SET1-1 F08 5 (1, 1, 2048, 3072)
imwrite :  ./denoised_img_F08_vs_F16_v2_crushed/SET1-1_F08_15.png
SET1-1_F08_16.png SET1-1 F08 6 (1, 1, 2048, 3072)
imwrite :  ./denoised_img_F08_vs_F16_v2_crushed/SET1-1_F08_16.png
SET1-1_F08_17.png SET1

# saving F32 image

In [69]:

img_dict = {}
# data_path = "/mnt/ssd/hyun/fbi-net/FBI-Denoiser/data/Samsung_SNU_1474x3010_aligned_ordered/" 
debug = True
path_list = ["/home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216"]
for data_path in path_list:
    print("=====",data_path, "=====")
    for set_num in sorted(os.listdir(data_path)):
        if set_num[0] == '.' :
            continue
        set_path = os.path.join(data_path,set_num)
        set_number = set_num.split(" ")[-1][:-1]
        set_num = f"{set_num[1:4]}{set_number}"
        print("=====",set_num, "=====")
        img_dict[set_num] = {}
        # if debug is True:
        #     print(set_num)
        f_num_list = os.listdir(set_path)
        print(f_num_list)
        for f_num in f_num_list:
            f_path = os.path.join(set_path,f_num)
            f_number = int(f_num[1:])
            f_num = f"F{f_number:02d}"
            # print(set_num,f_num)
            if f_num != 'F32' :
                continue
            img_dict[set_num][f_num] = None
            img_list = sorted(os.listdir(f_path))
            img_list = list(filter(lambda x : ".ipynb_checkpoints" not in x,img_list))
            
            for img_path in sorted(img_list):

                img_path = os.path.join(f_path,img_path)
                if debug is True:
                    print(f_num," & ",img_path)
                img = cv2.imread(img_path,cv2.IMREAD_GRAYSCALE) 
                # img_dict[set_num][f_num].append(img)
                img = np.expand_dims(img,axis=0)
                img = np.expand_dims(img,axis=0) / 255.

                if img_dict[set_num][f_num] is None:  
                    img_dict[set_num][f_num] = img
                else :
                    img_dict[set_num][f_num] = np.append(img_dict[set_num][f_num],img,axis=0)
                if debug is True:
                    print(set_num,f_num,img.shape)
                    
if debug is True:
    print(img_dict.keys())
    for set_num in img_dict.keys():
        print(img_dict[set_num].keys())
        for f_num in img_dict[set_num].keys():
            print(img_dict[set_num][f_num].shape)

===== /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216 =====
===== SET1-1 =====
['F32', 'F8', 'F64', 'F16']
F32  &  /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216/[SET 1-1]/F32/10_F32.png
SET1-1 F32 (1, 1, 2048, 3072)
F32  &  /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216/[SET 1-1]/F32/11_F32.png
SET1-1 F32 (1, 1, 2048, 3072)
F32  &  /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216/[SET 1-1]/F32/12_F32.png
SET1-1 F32 (1, 1, 2048, 3072)
F32  &  /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216/[SET 1-1]/F32/13_F32.png
SET1-1 F32 (1, 1, 2048, 3072)
F32  &  /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216/[SET 1-1]/F32/14_F32.png
SET1-1 F32 (1, 1, 2048, 3072)
F32  &  /home/hyunwoong/FBI-Denoiser/data/Samsung_tmp_dataset/Samsung+SNU+dataset+230216/[SET 1-1]/F32/15_F32.png
SET1-1 F32 (1, 1, 2048, 30

In [72]:
debug= True
# for fbi_weight_dir in sorted(glob("../weights/230207_FBI_Net*")):
for fbi_weight_dir in sorted(glob("../weights/230303_FBI_Net*F08*F16_MSE*")):
    target_y = fbi_weight_dir.split("_y_as_")[1][:3]
    if "x_as_" in fbi_weight_dir:
        target_x = fbi_weight_dir.split("_x_as_")[1][:3]
    else :
        target_x = "F#"
    dataset_version = 'v1' if "with" in fbi_weight_dir else 'v2'

    if target_y not in ['F08','F16']:
        continue
    print("======",target_x,"vs",target_y,dataset_version,f" ======")

    if debug is True:
        print(fbi_weight_dir)
        # print(median_filter_input,"median_filter_input")
    folder_name = f"./F32_img_crushed/"

    os.makedirs(folder_name,exist_ok=True)
    

    # DENOISE IMAGE
    denoised_img_dict = {}
    for set_num in img_dict.keys():
        denoised_img_dict[set_num] = {}
        for f_num in img_dict[set_num].keys():
            denoised_img_dict[set_num][f_num] = None
            if f_num != 'F32':
                continue
            
            for idx,img in enumerate(img_dict[set_num][f_num],start=1):
                
                print(set_num,f_num,idx,img.shape)
                
                denoised_img = img
                # print("after denoised",denoised_img.shape)
                if denoised_img_dict[set_num][f_num] is None:
                    denoised_img_dict[set_num][f_num] = denoised_img
                else :
                    denoised_img_dict[set_num][f_num] = np.append(denoised_img_dict[set_num][f_num], denoised_img,axis=0)
                filename = f"{set_num}_{f_num}_{idx:02d}.png"
                # if os.path.isfile(f"{folder_name}/{filename}") is False:
                print("imwrite : ",f"{folder_name}/{filename}")
                cv2.imwrite(f"{folder_name}/{filename}",denoised_img[0]*255)
                # apply median filter after denoising
                
            print(f"Denoising SET : {set_num}, f_num : {f_num} End")
            
    # np.save(f"./intermediate_result/{folder_name}_denoised_img_dict.npy",denoised_img_dict)

    gc.collect()


====== F08 vs F16 v2  ======
../weights/230303_FBI_Net_Grayscale_Samsung_SET050607080910individual_x_as_F08_y_as_F16_MSE_Affine_layers_x17_filters_x64_cropsize_256.w
SET1-1 F32 1 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_01.png
SET1-1 F32 2 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_02.png
SET1-1 F32 3 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_03.png
SET1-1 F32 4 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_04.png
SET1-1 F32 5 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_05.png
SET1-1 F32 6 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_06.png
SET1-1 F32 7 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_07.png
SET1-1 F32 8 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_08.png
SET1-1 F32 9 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_09.png
SET1-1 F32 10 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET1-1_F32_10.png
SET1-1 F32 11 (1, 2048, 3072)
imwrite :  ./F32_img_crushed//SET